In [1]:
import pandas as pd
import numpy as np
import joblib

# Chemins
TEST_PATH = "data/test.parquet"
EXAMPLE_PATH = "data/example_submission.csv"
MODEL_FILE = "xgb_final_optimized.pkl"
OUTPUT_FILE = "submission_xgb_gridsearch.csv" 

print("✅ Configuration chargée.")


✅ Configuration chargée.


In [2]:
# -------------------------------------
# 📥 Chargement des données test
# -------------------------------------

print("⏳ Chargement du fichier Test...")
test_df = pd.read_parquet(TEST_PATH)

# Conversion timestamps
test_df["ts"] = pd.to_datetime(test_df["ts"], unit="ms")
test_df["date"] = test_df["ts"].dt.date

T_test = test_df["ts"].max()
print(f"📅 Date de référence (T_test) : {T_test}")

test_users = test_df["userId"].unique()
print(f"👥 Utilisateurs à prédire : {len(test_users)}")


⏳ Chargement du fichier Test...
📅 Date de référence (T_test) : 2018-11-20 00:00:00
👥 Utilisateurs à prédire : 2904


In [3]:
# ============================================================
# 🏗️ RECONSTRUCTION DES FEATURES (identique au train)
# ============================================================

print("\n🏗️ Reconstruction des features Globales...")

# -------- GLOBAL FEATURES --------
global_feats = test_df.groupby("userId").agg({
    "ts": "max",
    "date": "nunique",
    "sessionId": "nunique",
    "length": "sum",
    "registration": "min"
}).reset_index()

global_feats.columns = ["userId", "last_ts", "n_active_days", "n_sessions", "total_listening_time", "registration_ts"]

# Convert dates
global_feats["registration_ts"] = pd.to_datetime(global_feats["registration_ts"], unit="ms")

# Derived features
global_feats["recency_days"] = (T_test - global_feats["last_ts"]).dt.days
global_feats["account_age_days"] = (T_test - global_feats["registration_ts"]).dt.days
global_feats["avg_daily_listen"] = global_feats["total_listening_time"] / (global_feats["account_age_days"] + 1)

print("✅ Global Features OK")

print("\n⏱️ Construction des features de fenêtres (7/14 jours)...")

def build_window_stats_test(df_all, T_ref, window_days, suffix):
    T_start = T_ref - pd.Timedelta(days=window_days)
    win = df_all[df_all["ts"] >= T_start]

    if win.empty:
        return pd.DataFrame({"userId": df_all["userId"].unique()})

    win_stats = win.groupby("userId").agg({
        "length": "sum",
        "sessionId": "nunique",
        "date": "nunique"
    }).reset_index()

    win_stats.columns = [
        "userId",
        f"listen_time_{suffix}",
        f"sessions_{suffix}",
        f"active_days_{suffix}"
    ]
    return win_stats

win_7d  = build_window_stats_test(test_df, T_test, 7,  "7d")
win_14d = build_window_stats_test(test_df, T_test, 14, "14d")

windows_test = pd.DataFrame({"userId": test_users})
for w in [win_7d, win_14d]:
    windows_test = windows_test.merge(w, on="userId", how="left")

windows_test = windows_test.fillna(0)

# Ratios
windows_test = windows_test.merge(
    global_feats[["userId", "total_listening_time"]],
    on="userId",
    how="left"
)

windows_test["ratio_listen_7d_14d"] = windows_test["listen_time_7d"] / (windows_test["listen_time_14d"] + 1)
windows_test["ratio_listen_7d_global"] = windows_test["listen_time_7d"] / (windows_test["total_listening_time"] + 1)

windows_test = windows_test.drop(columns=["total_listening_time"])

# -------- BEHAVIORAL FEATURES --------

print("\n📊 Reconstruction des features comportementales...")

page_counts = pd.pivot_table(
    test_df, index="userId", columns="page", values="ts",
    aggfunc="count", fill_value=0
).reset_index()

useful_pages = ["Thumbs Up", "Thumbs Down", "Roll Advert", "Error", "Upgrade", "Downgrade", "Add to Playlist"]
behavior_df = page_counts[["userId"] + [p for p in useful_pages if p in page_counts.columns]]

if "Thumbs Up" in behavior_df and "Thumbs Down" in behavior_df:
    behavior_df["satisfaction_ratio"] = behavior_df["Thumbs Up"] / (behavior_df["Thumbs Down"] + 1)

print("✅ Behavioral Features OK")

# -------- TREND FEATURES --------

print("\n📈 Reconstruction des trends...")

T_recent = T_test - pd.Timedelta(days=14)
recent = test_df[test_df["ts"] >= T_recent]

recent_stats = (
    recent.groupby("userId")["length"]
    .sum()
    .reset_index()
    .rename(columns={"length": "listen_time_recent"})
)

trends = global_feats[["userId", "avg_daily_listen"]].merge(recent_stats, on="userId", how="left").fillna(0)

trends["avg_daily_listen_recent"] = trends["listen_time_recent"] / 14
trends["trend_listening"] = trends["avg_daily_listen_recent"] / (trends["avg_daily_listen"] + 0.01)

print("✅ Trend Features OK")

# -------- DEVICE FEATURES --------

print("\n💻 Extraction des features techniques...")

last_agent = test_df.sort_values("ts").groupby("userId")["userAgent"].last().reset_index()

def flag(pattern):
    return last_agent["userAgent"].str.contains(pattern, case=False, na=False).astype(int)

last_agent["is_mac"]     = flag("Macintosh")
last_agent["is_windows"] = flag("Windows")
last_agent["is_linux"]   = flag("Linux")
last_agent["is_mobile"]  = flag("Mobile|iPhone|Android|iPad")
last_agent["is_firefox"] = flag("Firefox")
last_agent["is_chrome"]  = flag("Chrome")

tech_features = last_agent[["userId","is_mac","is_windows","is_linux","is_mobile","is_firefox","is_chrome"]]

print("✅ Device Features OK")


🏗️ Reconstruction des features Globales...
✅ Global Features OK

⏱️ Construction des features de fenêtres (7/14 jours)...

📊 Reconstruction des features comportementales...


/var/folders/hw/wlt97vgx76x25gg48pgl0flh0000gn/T/ipykernel_99305/2161045420.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  behavior_df["satisfaction_ratio"] = behavior_df["Thumbs Up"] / (behavior_df["Thumbs Down"] + 1)


✅ Behavioral Features OK

📈 Reconstruction des trends...
✅ Trend Features OK

💻 Extraction des features techniques...
✅ Device Features OK


In [4]:
# ============================================================
# 🧩 FUSION FINALE DES FEATURES
# ============================================================

print("\n🧩 Fusion finale...")

X_test = pd.DataFrame({"userId": test_users})

X_test = (
    X_test
    .merge(global_feats, on="userId", how="left")
    .merge(behavior_df, on="userId", how="left")
    .merge(trends[["userId","trend_listening"]], on="userId", how="left")
    .merge(windows_test, on="userId", how="left")
    .merge(tech_features, on="userId", how="left")
    .fillna(0)
)

# Sauvegarde des userId pour la submission
userId_col = X_test["userId"]


# -------------------------
# ❌ SUPPRESSION des colonnes datetime
# -------------------------
cols_to_drop = ["userId", "last_ts", "registration_ts"]
X_test = X_test.drop(columns=[c for c in cols_to_drop if c in X_test.columns])

print(f"Shape X_test après nettoyage datetime : {X_test.shape}")


🧩 Fusion finale...
Shape X_test après nettoyage datetime : (2904, 29)


In [5]:
# CELLULE AJOUTER POUR TESTER L'ENSEMBLE

# -------------------------------------
# 3. Chargement modèle XGB
# -------------------------------------
print("\n📥 Chargement modèle XGBoost...")

model = joblib.load(MODEL_FILE)
best_threshold = 0.5  # ou autre seuil si tu en as un

print(f"🔥 Seuil utilisé : {best_threshold:.2f}")

# Alignement colonnes
expected_cols = model.get_booster().feature_names
X_test = X_test.reindex(columns=expected_cols, fill_value=0)

print("📐 Colonnes alignées avec le modèle.")
print("X_test final :", X_test.shape)



📥 Chargement modèle XGBoost...
🔥 Seuil utilisé : 0.50
📐 Colonnes alignées avec le modèle.
X_test final : (2904, 29)


In [6]:
# ============================================================
# 🔮 PREDICTION
# ============================================================

print("\n🔮 Génération des prédictions...")

probs = model.predict_proba(X_test)[:, 1]
preds = (probs >= best_threshold).astype(int)

print(f"📊 Taux de churn prédit : {preds.mean():.2%}")

preds = (probs >= best_threshold).astype(int)

print(f"📊 Taux de churn prédit : {preds.mean():.2%}")

submission = pd.DataFrame({
    "id": userId_col,
    "target": preds
})



# print("🔮 Génération des prédictions...")

# # 1. Calcul des probabilités
# probs = xgb_model.predict_proba(X_test)[:, 1]

# # 2. Application du SEUIL OPTIMAL (celui trouvé dans notebook 02)
# #preds = (probs >= best_threshold).astype(int)

# MANUAL_THRESHOLD = 0.55 

# print(f"🔧 Test avec seuil manuel : {MANUAL_THRESHOLD}")
# preds = (probs >= MANUAL_THRESHOLD).astype(int)

# # Vérifie le taux avant de sauvegarder
# print(f"Nouveau taux de churn prédit : {preds.mean():.2%}")

# # 3. Création du fichier de soumission
# submission = pd.DataFrame({
#     "id": userId_col,
#     "target": preds
# })

# # Vérification du format avec l'exemple (si disponible)
# try:
#     example = pd.read_csv(EXAMPLE_PATH)
#     example["id"] = example["id"].astype(str)
#     submission["id"] = submission["id"].astype(str)
    
#     # On garde seulement les IDs demandés dans l'exemple, dans le bon ordre
#     final_submission = example[["id"]].merge(submission, on="id", how="left")
    
#     # Remplir les éventuels trous par 0 (sécurité)
#     final_submission["target"] = final_submission["target"].fillna(0).astype(int)
    
#     print("✅ Alignement avec example_submission.csv réussi.")
# except FileNotFoundError:
#     print("⚠️ example_submission.csv non trouvé, on sauvegarde tel quel.")
#     final_submission = submission

# # Stats
# n_churn = final_submission["target"].sum()
# total = len(final_submission)
# print(f"\n📊 Résultat : {n_churn} churners détectés sur {total} utilisateurs.")
# print(f"   Taux de churn prédit : {n_churn/total:.2%}")

# # Sauvegarde
# final_submission.to_csv(OUTPUT_FILE, index=False)
# print(f"💾 Fichier sauvegardé : {OUTPUT_FILE}")


🔮 Génération des prédictions...
📊 Taux de churn prédit : 53.24%
📊 Taux de churn prédit : 53.24%


In [7]:
# ============================================================
# 📤 ALIGNEMENT AVEC example_submission.csv
# ============================================================

try:
    example = pd.read_csv(EXAMPLE_PATH)
    example["id"] = example["id"].astype(str)
    
    final_submission = example[["id"]].merge(
        submission.assign(id=submission["id"].astype(str)),
        on="id",
        how="left"
    )

    final_submission["target"] = final_submission["target"].fillna(0).astype(int)
    print("✅ Alignement avec example_submission.csv réussi.")

except FileNotFoundError:
    print("⚠️ Pas d'exemple trouvé → fichier envoyé tel quel.")
    final_submission = submission

# ============================================================
# 💾 SAUVEGARDE
# ============================================================

final_submission.to_csv(OUTPUT_FILE, index=False)
print(f"\n💾 Fichier final sauvegardé : {OUTPUT_FILE}")
print("🎉 Submission prête (XGB only) !")

✅ Alignement avec example_submission.csv réussi.

💾 Fichier final sauvegardé : submission_xgb_gridsearch.csv
🎉 Submission prête (XGB only) !
